In [3]:
import requests, json
import pandas as pd
import numpy as np
from datetime import date, timedelta

headers = {
'Accept': 'application/x-google-protobuf',
'Authorization': 'apikey vy2rFNXj4MRr2fuPWkB9etPZlz2yiD7NoWHl',
}

df_res = pd.DataFrame()
all_res = []

def get_carspaces(facility,date):

    url = f'https://api.transport.nsw.gov.au/v1/carpark/history?facility={facility}&eventdate={date}'

    response = requests.get(url, headers=headers, stream=True)
    
    json_string = json.loads(response.content)

    list_of_available = []
    list_timestamp = []
    df_car_spaces = pd.DataFrame()
    for x in range(len(json_string)):
        total = json_string[x]['occupancy']['total']
        timestamp = json_string[x]['MessageDate']
        spots = json_string[x]['spots']
        list_of_available.append( int(spots) - int(total))
        list_timestamp.append( timestamp)
    
    df_car_spaces['Timestamp'] = list_timestamp
    df_car_spaces['Available'] = list_of_available
    all_res.append(df_car_spaces)

# Parse date range
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Set start and end datetimes of 1 month 
start_date = date(2022, 3, 11)
end_date = date.today()
for single_date in daterange(start_date, end_date):
    #print(single_date.strftime("%Y-%m-%d"))
    # 14 - West Ryde Car Park
    get_carspaces(14,single_date)

df_res = pd.concat(all_res)
df_res.to_csv('../Resources/carspaces_data.csv')